# L5b: Single Asset Geometric Brownian Motion Models
In this lecture, we introduce our first continuous-time model for asset prices, the Geometric Brownian Motion (GBM) model. This model is __widely__ used in finance due to its mathematical tractability and ability to capture key features of asset price dynamics.

> __Learning Objectives:__
>
> By the end of this lecture, you will be able to:
> 
> * **Understand Geometric Brownian Motion (GBM) as a continuous-time asset price model.** We introduce GBM as our first continuous stochastic process for modeling share prices, including its mathematical foundation and analytical solution that makes it tractable for both theory and simulation.
> * **Estimate GBM parameters from historical data using multiple methods.** We learn how to calculate drift and volatility parameters from real market data using both averaging techniques and linear regression approaches to fit the model to observed price movements.
> * **Assess GBM model performance and calculate NPV-based return probabilities.** We evaluate how well GBM captures key market stylized facts like return distributions and autocorrelation, then use net present value calculations to compute probabilities of achieving target returns for continuous price processes.

Let's get started!
___

## Examples
> [▶ Let's Explore N-Ary Lattice Models](CHEME-5660-L5a-Example-N-Aray-Lattice-Fall-2025.ipynb). In this example, we extend the binomial lattice model to an n-ary lattice model, where the share price can move to many possible values at each time step (not just up or down). In the limit of many possible values, we can approximate a continuous distribution of share prices. Wow!

> [▶ Build a Single Asset Geometric Brownian Motion Model](CHEME-5660-L5b-Example-Parameters-SAGBM-Fall-2025.ipynb). In this example, we build a single asset geometric Brownian motion model, where the share price follows a stochastic process with a constant drift and volatility. We can use this model to simulate future price paths and analyze the expected price and uncertainty. 

> [▶ Implement an NPV-Based Trade Rule Using GBM Price Modeling](CHEME-5660-L5b-Example-GBM-NPV-TradeRule-Fall-2025.ipynb). In this example, we implement a trade rule based on the Net Present Value (NPV) where the future share price is modeled using a Geometric Brownian Motion (GBM). 
___

## Company Profile: Jane Street
[Jane Street](https://www.janestreet.com) is a privately held quantitative trading firm with approximately 3,000 employees. It trades across asset classes (equities/ETFs, options, bonds, FX, and more) on 200+ venues in 45 countries, and is a major market maker in ETFs. 

> **Business model.** Jane Street (JS) principally deploys its own capital (proprietary trading) while providing liquidity to __institutions__ (e.g., as an ETF authorized participant/lead market maker). The firm is deeply tech-led (functional programming, ML, hardware acceleration) and emphasizes collaborative, firm-wide risk management. [Let's look at what the interns say about Jane Street.](https://www.janestreet.com/join-jane-street/internships/)

**How Jane Street contrasts with Citadel Securities (CS).**
While both Jane Street and Citadel Securities are leading quantitative trading firms, they have some notable differences:

* **Client posture:** [Citadel Securities (CS)](https://www.citadelsecurities.com) handles approximately 40% of U.S daily retail trade volume for equity/option order flow. On the other hand, Jane Street is primarily proprietary and institution-facing (does handle some retail ETF flow). In June 2025: Citadel Securities handled 38.3% of US retail traffic, while Jane Street handled 11.6%. This data is from [Rule 605 reports](https://www.sec.gov/rule-release/34-43590) filed with the SEC. Here is [Citadel Securities Rule 605 filings](https://www.citadelsecurities.com/rule-605-606-statements/) and [Jane Street Rule 605 filings](https://www.janestreet.com/execution-quality-reports/). The field descriptions of the Rule 605 reports can be found [here](https://www.sec.gov/files/oes_data-reports-sec_rule_605.pdf).
* **Product emphasis:** Jane Street is especially synonymous with ETFs, while Citadel Securities is most identified with U.S. equities/options/FX market making and order flow.
* **Scale & performance.** Citadel Securities took outside capital [in 2022 at a 22B USD valuation](https://www.reuters.com/business/finance/citadel-securities-valued-22-bln-after-investment-sequoia-paradigm-2022-01-11/?utm_source=chatgpt.com). While Jane Street remains privately held with limited external disclosures, however, they disclosed larger trading revenue in recent years and exceptionally high trading volumes, particularly in ETFs. Jane Street expanded materially in Asia in 2024–2025 (e.g., Hong Kong).
* JS optimizes for correctness/composability (functional programming [in OCaml](https://ocaml.org), proofs, internal seminars). CS optimizes for industrial reliability at scale (pipelines, hardware, exchange microstructure). Different flavors of intensity.

So it's hard to say which firm is better. Both places are rigorously selective, intensely quantitative, and very tech-forward. From an analytical perspective, here's how one might choose between them:

> __Which should I choose (GPT-5 Thinking):__
> 
> Pick Jane Street if you think: “I’d rather prove properties and build elegant functional programming systems that trade across many venues; I care that my abstractions are beautiful and safe. Optimization axis: JS leans more toward correctness/robustness, model nuance."
>
> Pick Citadel Securities if you think: “Give me the biggest pipes in the market and the tightest latency/scale constraints; I want my math to survive brutal production realities and move national-level order flow. Optimization axis: CS leans more toward throughput/latency, capacity, reliability under load.”

Citadel Securities skews “hard edges at scale,” Jane Street skews “rigor + collaboration.” Both are intense; they just express intensity differently. Pick your flavor.
___


## Concept Review: N-Ary Tree Models
One obvious limitation of the binomial lattice model is that it only allows for two possible price movements at each time step (up or down). In reality, asset prices can exhibit a wider range of behaviors. 

> __Idea__: If two possible futures are not enough, we can extend the binomial lattice model to an N-ary model, e.g., a trinomial lattice model (three possible price movements: up, down, or unchanged) or even more complex models with multiple price levels at each time step. Let's explore this idea.

Let's do a quick motivational example.

> __Example__
>
> [▶ Let's Explore N-Ary Lattice Models](lectures/week-5/L5a/CHEME-5660-L5a-Example-N-Aray-Lattice-Fall-2025.ipynb). In this example, we extend the binomial lattice model to an n-ary lattice model, where the share price can move to many possible values at each time step (not just up or down). In the limit of many possible values, we can approximate a continuous distribution of share prices. Wow!

Wow! That example was cool, and it lays the foundation for today's lecture where we explore continuous stochastic models of share price.
___

## Single Asset Geometric Brownian Motion (GBM)
Geometric Brownian motion (GBM) is a stochastic differential equation describing the share price $S(t)$ 
as a random walk with drift and volatility proportional to $S(t)$:
$$
\begin{align*}
\frac{dS\left(t\right)}{S(t)} &= \mu\,{dt}+\sigma\,{dW(t)}\\
\end{align*}
$$
Here, $\mu\in\mathbb{R}$ denotes a constant drift (growth) parameter, $\sigma>0$ is a constant __volatility__ parameter, and $dW$ represents a Wiener noise (randomness) process:

> __Wiener Process__
> 
> A Wiener Process is a real-valued continuous-time stochastic process named after Norbert Wiener for the study of one-dimensional Brownian motion. A Wiener process is a continuous one-dimensional stochastic process $\left\{W\left(t\right), 0\leq{t}\leq{T}\right\}$ with the following properties:
>
> * The noise vanishes at zero: $W\left(0\right)$ = $0$ with probability $1$
> * The increments $\left\{W(t_{1}) - W(t_{o}),\dots, W(t_{k}) - W(t_{k-1})\right\}$ are independent for any $k$ and $0\leq{t_{o}}< t_{1} < \dots < t_{k} \leq{T}$}
> * The increment $(W(t) - W(s))\sim~\mathcal{N}\left(0,t-s\right)$ for any $0\leq{s}< t \leq{T}$, where $\mathcal{N}\left(0,t-s\right)$ denotes a normally distributed random variable with mean $0$ and variance $t - s$. Thus, we can write $W(t) - W(s) = \left(\sqrt{t-s}\right)\;Z(0,1)$, where $Z(0,1)$ is a standard normal random variable.
> 

We can solve the [GBM stochastic differential equation using Itô calculus](HEME-5660-L5b-GBM-Solution-Derivation-Fall-2025.ipynb). One of the really nice features of this model is that it has the (cumulative) analytical solution:
$$
\boxed{
\begin{align*}
S_{t} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(t - t_{0}\right) + (\sigma\sqrt{t-t_{0}})\;{Z_{t}(0,1)}\Biggr]\\
\end{align*}}
$$
where $S_{0}$ is the initial share price at $t_{0}$, and $Z_{t}(0,1)$ is a standard normal random variable drawn at time $t$. The expectation and variance of the cumulative GBM model is given by:
$$
\begin{align*}
\mathbb{E}\left(S_{t}\right) &= S_{0}\;\exp\left(\mu(t-t_{0})\right)\\
\text{Var}\left(S_{t}\right) &= S_{0}^{2}e^{2\mu\cdot(t-t_{0})}\left[e^{\sigma^{2}(t-t_{0})} - 1\right]
\end{align*}
$$
These expressions show that the expected price grows exponentially with rate $\mu$, and the variance in price also grows exponentially with time, influenced by both the drift $\mu$ and volatility $\sigma$.

### Discrete Time GBM Model
The expressions for __continuous time__ $t$ are theoretically interesting, but not directly applicable to practical scenarios, e.g., we don't have continuous price data, instead we have it at discrete time intervals. For __discrete time__, e.g., the price at discrete time samples of size $\Delta{t}$, we can write the discrete time GBM model as (assuming $t_{0} = 0$):
$$
\boxed{
\begin{align*}
S_{i} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)i\Delta{t} + (\sigma\sqrt{i\Delta{t}})\;{Z_{i}(0,1)}\Biggr]\quad\text{where}\,i=1,2,\dots, N\\
\end{align*}}
$$

In addition, we sometimes just want to predict the price one-step ahead, i.e., at time $t + \Delta{t}$ given the price at time $t$. In this case, we can write the GBM model as:
$$
\boxed{
\begin{align*}
S_{t+\Delta{t}} &= S_{t}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\Delta{t} + (\sigma\sqrt{\Delta{t}})\;{Z_{t+\Delta{t}}(0,1)}\Biggr] \\
\end{align*}}
$$

### Monte Carlo Simulation of GBM
We can use the discrete time GBM model to simulate many possible price paths, i.e., possible alternative future prices, using Monte Carlo simulation. 

> __Idea:__ The basic idea is to generate many samples from the $Z_{i}(0,1)$ random variable, and use these samples to generate many possible price paths. We can then analyze the distribution of these price paths to understand the expected price and the uncertainty (variance) in the price.

Let's look at some pseudo-code for the Monte Carlo simulation of the GBM model.

__Initialize:__ Given the initial price $S_{0}$, drift $\mu$, volatility $\sigma$, time step $\Delta{t}$, and number of steps $N$.
Initialize an array to store the simulated prices $\mathbf{X}\in\mathbb{R}^{N \times M}$, where $M$ is the number of simulations (sample paths).

For each simulation $j = 1$ to $M$ (number of simulations) __do:__
1. Set the initial price for this simulation: $X_{1,j} \gets S_{0}$
2. For each time step $i = 2$ to $N$ __do:__
   - a. Generate a random sample from the standard normal distribution: $Z_{i,j} \sim \mathcal{N}(0,1)$
   - b. Compute the price at time step $i$ using the GBM formula:
   $$
   X_{i,j} \gets X_{i-1,j} \cdot \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\Delta{t} + (\sigma\sqrt{\Delta{t}})\;{Z_{i,j}}\Biggr]
   $$

This process generates $M$ different price paths (columns) of $N$ time steps (rows), each representing a possible future trajectory of the asset price under the GBM model. We've implemented this logic in [the `sample(…)` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.sample-Tuple{MyGeometricBrownianMotionEquityModel,%20NamedTuple}).

___

## Estimation of GBM Parameters
The parameters in the GBM model embody the risk versus return trade-off of the asset.

> __Parameters__
> 
> * __Reward:__ The __drift__ parameter $\mu\in\mathbb{R}$ (units: inverse years) represents the expected return of the asset over a given time period  in the absence of randomness. It is the reward for holding the asset.
> * __Risk:__ The __volatility__ parameter $\sigma\in\mathbb{R}_{>0}$ (units: inverse years to the 1/2 power) represents the standard deviation of the asset's growth rate, capturing the asset's price uncertainty. The volatility term $\sigma$ is always positive, i.e., $\sigma > 0$. We can think of volatility as the risk associated with holding the asset.

We can estimate the drift $\mu$ and volatility $\sigma$ parameters of a GBM model using historical price data. 

### Volatility
Let's start with the volatility term $\sigma$. Suppose we price observations from $0\rightarrow{T}$. From these observations, we can construct a time series of length $N-1$ of continuously compounded growth rates $\left\{\mu_1,\mu_2,\ldots,\mu_{N-1}\right\}$ for a stock. The sample standard deviation of the continuously compounded growth rate (our risk measure), is given by:
$$
\boxed{
\begin{align*}
\sigma_{\mu} &= \underbrace{\sqrt{\frac{1}{N - 2}\sum_{t=2}^{N}\left(\mu_t - \mu^{\prime}\right)^2}}_{\text{volatility of the growth rate}} \\
\end{align*}}
$$
where $\mu^{\prime} = \frac{1}{N-1}\sum_{t=2}^{N}\mu_t$ is the sample mean of the $N-1$ growth observations. The denominator $N-2$ represents the degrees of freedom for the sample standard deviation: $(N-1) - 1 = N-2$. We'll typically compute the standard deviation using [the `std(...)` method exported by the `Statistics.jl` package](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.std).



### Drift
Next, let's move on to the drift term $\mu$ (units: inverse years). The drift term $\mu$ represents the expected return of the asset over a given time period. There are two common methods to estimate the drift term $\mu$ of the GBM model.

> __Estimating the Drift Term ($\mu$):__
>
> * __Method 1: Average Log Growth__: The first, is to compute the log growth rates of the asset prices over a historical period and then take the average of these log growth rates. This method is straightforward and provides a simple estimate of the drift.
> * __Method 2: Linear Regression__: The second method involves performing a linear regression on the logarithm of the asset prices over time __assuming the noise term is zero__. By fitting a linear model to the log prices, we can estimate the slope of the line, which corresponds to the drift term $\mu$. This method can be more robust, especially when dealing with noisy data, e.g., we can use robust regression techniques to mitigate the impact of outliers. 
> 
> Let's check out the ideas behind the linear regression method.

#### Linear Regression
Let's assume the we have a price dataset for some firm that we've collected everyday for some time period, $\left\{S_{0}, S_{1}, \ldots, S_{T}\right\}$. We can denote the price at time step $i$ as $S_{i}$, where $i=0,1,2,\ldots,T$. For us, the time step size is $\Delta{t}$ (e.g., 1 day), however, we can also have shorter or longer time steps. Then, we can write the cumulative GBM solution at time $t$ as:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\quad{i = 1,2,\ldots, N}\\
\end{align*}
$$
where $T = N\Delta{t}$, and $Z\sim\mathcal{N}(0,1)$ is a standard normal random variable.
In this method, we assume that the noise term is zero, i.e., $\sigma = 0$. This simplifies the GBM solution to:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\left(\mu\cdot{i\Delta{t}}\right)\quad\Longrightarrow\text{Taking the natural logarithm of both sides gives:}\\
\ln(S_{i}) &= \ln(S_{0}) + \mu\cdot{i\Delta{t}}\quad\quad{i=1,2,\ldots, N}\\
\end{align*}
$$
where $\ln(S_{0})$ is a constant (share price at time $t=0$, y-intercept). This equation represents a linear relationship between $\ln(S_{i})$ (observation) and $i\Delta{t}$ (independent variable), where the slope of the line is $\mu$ (units: inverse years). Given the price observations ${S_{1},\ldots,S_{N}}$, we can construct an __overdetermined__ system of equations:
$$
\begin{align*}
\hat{\mathbf{X}}\mathbf{\theta} + \epsilon &= \mathbf{y}\\
\end{align*}
$$
where $\mathbf{\theta}$ is the parameter vector we want to estimate (the y-intercept $\ln(S_{0})$ and slope $m = \mu$), $\epsilon\sim\mathcal{N}(0,\sigma_{\epsilon}^{2})$ is an error model, $\mathbf{X}$ is the $N\times{2}$ design matrix, and $\mathbf{y}$ is the $N\times{1}$ observation vector. The design matrix $\mathbf{X}$ is given by:
$$
\begin{align*}
\hat{\mathbf{X}} &= \begin{bmatrix}
1 & 1\Delta{t} \\
1 & 2\Delta{t} \\
\vdots & \vdots \\
1 & N\Delta{t}
\end{bmatrix}\\
\end{align*}
$$
while the observation vector $\mathbf{y}$ is given by:
$$
\begin{align*}
\mathbf{y} &= \begin{bmatrix}
\ln(S_{1}) \\
\ln(S_{2}) \\
\vdots \\
\ln(S_{N})
\end{bmatrix}\\
\end{align*}
$$
The unknown (expected) parameter vector estimate $\hat{\mathbf{\theta}}$ has an analytical solution given by the normal equations:
$$
\boxed{
\begin{align*}
\hat{\mathbf{\theta}} &= (\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}})^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}\quad\blacksquare\\
\end{align*}}
$$

Let's do an example where we estimate the drift and volatility parameters of a GBM model using historical price data.

> __Example__
> 
> [▶ Build a Single Asset Geometric Brownian Motion Model](CHEME-5660-L5b-Example-Parameters-SAGBM-Fall-2025.ipynb). In this example, we build a single asset geometric Brownian motion model, where the share price follows a stochastic process with a constant drift and volatility. We can use this model to simulate future price paths and analyze the expected price and uncertainty. 
___

## Stylized Facts
The GBM model is our second method to simulate asset prices. With any model, we want to ensure that it captures the key features of real-world asset prices. Thus, let's ask ourselves: Does the GBM model capture the stylized facts of asset prices?

__Stylized Facts of Asset Prices:__

* __Heavy (also called fat) tailed distribution__: Stock growth rates (returns) often exhibit a distribution with heavier tails than would be expected under a normal distribution. This means that extreme price movements are more likely than would be predicted by a normal distribution.
* __Absence of Autocorrelation__: Autocorrelation refers to the tendency of growth rates (returns) to correlate with past returns over time. Autocorrelation suggests predictability in growth rates (returns), which traders could exploit. On the other hand, a random walk is uncorrelated with itself. We expect a random walk with occasional jumps.
* __Volatility clustering__: Stock growth rates (returns) tend to be more volatile during specific periods and less volatile during others. This phenomenon is known as volatility clustering, suggesting that large price movements are more likely to be followed by other large moves, and other small moves follow small moves.

To explore this question, let's start by computing the growth rates of the GBM model:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\quad{i = 1,2,\ldots,N}\\
\frac{S_{i}}{S_{0}} &= \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\\
\ln\left(\frac{S_{i}}{S_{0}}\right) &= \left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\\
\underbrace{\left(\frac{1}{\Delta{t}}\right)\ln\left(\frac{S_{i}}{S_{0}}\right)}_{= g_{i,0}} &= \left(\mu-\frac{\sigma^{2}}{2}\right)i + \sigma\left(\sqrt{\frac{i}{\Delta{t}}}\right)\;{Z}\\  
\end{align*}
$$
where $g_{i,0}$ is the continuously compounded (multistep) growth rate from time $0\rightarrow{i\Delta{t}}$ (we use the subscript $i,0$ to denote the growth rate from time $0$ to time $i\Delta{t}$, and $g$ to avoid confusion with the drift parameter $\mu$).
Putting this all together, we have the continuously compounded (multistep) growth rate from time $0\rightarrow{i\Delta{t}}$:
$$
\boxed{
\begin{align*}
g_{i,0} &= \left(\mu-\frac{\sigma^{2}}{2}\right)i + \sigma\left(\sqrt{\frac{i}{\Delta{t}}}\right)\;{Z_{i}(0,1)}\quad\text{where}\;{i=1,2,3,\ldots, N}\\
\end{align*}}
$$
The $Z_{i}(0,1)$ term is a standard normal random variable at time index $i$. Thus, the (multistep) growth rate $g_{i,0}$ is normally distributed with mean $\left(\mu-\frac{\sigma^{2}}{2}\right)i$ and variance $\sigma^{2}\left(\frac{i}{\Delta{t}}\right)$. 

Wow! this already gives us some clues about whether the GBM model captures the stylized facts of asset prices.


> __Heavy-Tailed Returns:__ The single step and multistep growth rates $g_{i,0}$ are normally distributed with mean $\left(\mu-\frac{\sigma^{2}}{2}\right)i$ and variance $\sigma^{2}\left(\frac{i}{\Delta{t}}\right)$. Thus, the GBM model __will not capture__ the heavy-tailed nature of asset returns. 

> __Volatility Clustering:__ While the volatility term $\sigma\left(\sqrt{\frac{i}{\Delta{t}}}\right)$ increases with time step $i$, the volatility parameter $\sigma$ itself remains constant. Thus, there is no mechanism in the GBM model to capture volatility clustering, i.e., the alternation between high and low volatility periods.

Finally, let's consider the autocorrelation of the (single step) growth rates $g_{1,0}$. Let:
$$
\begin{align*}
X_t \equiv g_{1,0}(t)
& = \left(\mu-\frac{\sigma^{2}}{2}\right)\;+\;\sigma\sqrt{\frac{1}{\Delta t}}\;Z_t,
\end{align*}
$$
with ${Z_t}$ i.i.d. $\mathcal N(0,1)$. Write $a=\mu-\tfrac{\sigma^{2}}{2}$ and $b=\sigma\sqrt{1/\Delta t}$, so $X_t=a+bZ_t$. Then, the autocovariance at lag $\tau\ge 1$ is given by:
$$
\begin{align*}
\mathrm{Cov}(X_t,X_{t+\tau})
= \mathrm{Cov}(a+bZ_t,\;a+bZ_{t+\tau})
= b^2\,\mathrm{Cov}(Z_t,Z_{t+\tau})
= 0,
\end{align*}
$$
because $Z_t$ and $Z_{t+\tau}$ are independent (from our definition of the noise process). 
Hence the autocorrelation function is given by:
$$
\begin{align*}
\rho_X(\tau)=\frac{\mathrm{Cov}(X_t,X_{t+\tau})}{\mathrm{Var}(X_t)}=0,\qquad \tau\ge 1,
\end{align*}
$$
with $\mathrm{Var}(X_t)=b^2=\sigma^2/\Delta t$.

> __Absence of Autocorrelation:__ The autocorrelation function $\rho_X(\tau)$ is zero for all lags $\tau\ge 1$. Thus, the GBM model __does capture__ the absence of autocorrelation in single-step asset growth rates (returns). So, the GBM model captures this stylized fact!

___

## GBM Trade Rule
Let's develop an expression that allows us to compute trade entry and exit rules based on the net present value (NPV) of a trade.

> __Scenario:__ Suppose we purchase $n_{o}$ shares of ticker `XYZ` at time $t=0$ (today) for $S_{0}$ USD/share.
> Then, sometime later, at $T=i\Delta{t}$, we sell all $n_{o}$ shares at a share price of $S_{T}$ USD/share, where $i\geq{0}$ is the number of time steps (integer) and $\Delta{t}$ is the time step (e.g., one trading day written in units of years).


This trade is a __long position__ because we profit if the share price goes up, i.e., $S_{T} > S_{0}$. Further, we can model it as an __abstract asset__ that has two cash flow events: the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit). The net present value (NPV) of this trade (written from our perspective) will have two cash flow events, the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit):
$$
\begin{align*}
\texttt{NPV}(\bar{r}, T) &= \underbrace{-n_{o}\;{S_{0}}}_{\text{entry (now)}} + \underbrace{n_{o}\;{S_{T}}\;\mathcal{D}_{T,0}^{-1}(\bar{r})}_{\text{exit (future)}}\quad\Longrightarrow\text{divide by initial investment}\;n_{o}\;{S_{0}}\\
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}} &= \frac{S_{T}\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - S_{0}}{S_{0}}\\
\underbrace{\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}}}_{\text{fractional return}} &= \left(\frac{S_{T}}{S_{0}}\right)\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare
\end{align*}
$$
where $\mathcal{D}^{-1}_{T,0}(\bar{r})$ is the (inverse) continuous discount factor for period $0\rightarrow{T}$ assuming a (constant) discount rate $\bar{r}$. 

Unlike last lecture, we used a binomial lattice models, let's now assume that the share price $S_{T}$ follows a GBM model. Then, we can substitute the GBM solution for $S_{T}$ into our NPV expression and simplify gives:
$$
\boxed{
\begin{align*}
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}{S_{0}}} &= \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)T + (\sigma\sqrt{T})\;{Z(0,1)}\Biggr]\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare\\
\end{align*}}
$$

Let's take a closer look at this NPV expression, in the context of short versus long holding periods, given our geometric Brownian motion model for the share price $S_{T}$.

### Positive Trade Exit Probability
Unlike our binomial lattice model from last lecture, the NPV expression is a __probability distribution function__ (pdf) because the share price $S_{T}$ is a __continuous random variable__. However, we can still compute the probability that our trade meets some desired fractional return threshold $\texttt{NPV}(\bar{r}, T)/n_{o}S_{0} > r_{\star}$, given a holding period $T$ and discount rate $\bar{r}$.

Let's dig into this a bit more. Let
$$
X \equiv \left(\mu-\frac{\sigma^{2}}{2}\right)T + \sigma\sqrt{T}\;Z,\qquad Z\sim \mathcal{N}(0,1),
$$
which implies
$$
\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}=e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1.
$$
We want the scaled NPV to be (strictly) greater some specified fraction return $r_{\star}>0$, i.e., $\texttt{NPV}(\bar r,T)/n_o S_0> r_\star$. Thus,
$$
\begin{align*}
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
& = \mathbb{P}\!\left(e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1> r_\star\right)\\
& = \mathbb{P}\!\left(e^{X}> (1+ r_\star)\mathcal{D}_{T,0}(\bar r)\right)\\
& = \mathbb{P}\left(X>\ln\big((1+r_\star)\mathcal{D}_{T,0}(\bar r)\big)\right)
\end{align*}
$$
Ok, so here is the trick to this type of probability calculation. We need to write the probability expression in terms of the cumulative distribution function (CDF) of the standard normal random variable $Z$. To do this, we standardize the random variable $X$. Let $m=(\mu-\tfrac{1}{2}\sigma^2)T$ and $v=\sigma^2 T$. Then, solve $X$ for $Z$:
$$
\mathbb{P}\!\left(X>\ln((1+r_\star)\mathcal{D}_{T,0}(\bar r))\right)
= \mathbb{P}\!\left(\underbrace{\frac{X-m}{\sigma\sqrt{T}}}_{Z\sim\mathcal{N}(0,1)}>\frac{\ln((1+r_\star)\mathcal{D}_{T,0}(\bar r))-m}{\sigma\sqrt{T}}\right)
= 1-\Phi\!\left(\frac{\ln((1+r_\star)\mathcal{D}_{T,0}(\bar r))-m}{\sigma\sqrt{T}}\right),
$$
or equivalently
$$
\boxed{\;
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
= 1 - \Phi\!\left(\frac{\ln((1+r_\star)\mathcal{D}_{T,0}(\bar r))-m}{\sigma\sqrt{T}}\right)\quad\blacksquare}
$$

where $\Phi(\cdot)$ is the cumulative distribution for a standard normal random variable. If we hold the $n_{o}$ shares for only a __short time__, e.g., a few days or even a few weeks, for realistic choices of the discount rate $\bar{r}$, the discount factor $\mathcal{D}_{T,0}(\bar{r})\approx{1}$, which simplifies the probability expression.

Let's do an example.

> __Example__
>
> [▶ Implement an NPV-Based Trade Rule Using GBM Price Modeling](CHEME-5660-L5b-Example-GBM-NPV-TradeRule-Fall-2025.ipynb). In this example, we implement a trade rule based on the Net Present Value (NPV) where the future share price is modeled using a Geometric Brownian Motion (GBM). 

___

## Summary
In this lecture, we explored Geometric Brownian Motion as a fundamental continuous-time model for asset price dynamics and developed practical trading applications using probabilistic analysis.

> __Key Takeaways__
> * **GBM provides a mathematically tractable foundation for continuous price modeling.** The model captures asset prices through a stochastic differential equation with constant drift and volatility parameters, offering an analytical solution that enables both theoretical analysis and Monte Carlo simulation of future price paths.
>
> * **Parameter estimation requires careful analysis of historical return data.** We learned two approaches for estimating model parameters: direct calculation from return statistics for volatility, and both averaging and linear regression methods for drift estimation, each with their own advantages for different data characteristics.
>
> * **GBM enables probabilistic trading decisions but reveals model limitations.** We derived NPV-based trade rules that calculate exit probabilities for long positions, while discovering that GBM captures return autocorrelation patterns but fails to reproduce heavy-tailed distributions and volatility clustering seen in real markets.

___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___